In [ ]:
!pip install langchain bitsandbytes accelerate langchain_community PyPDF opendatasets sentence-transformers faiss-gpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.4 MB/s eta 0:00:00


# Overview:

---


This project develops a question answering (QA) system for PDF documents, utilizing FAISS for indexing and efficient text retrieval. It employs the Mistral-7B-Instruct model, fine-tuned for instructional text comprehension, to generate answers based on user queries. The system extracts text from PDFs, embeds it using HuggingFace's embeddings, and stores them in FAISS for fast retrieval. Users can ask questions about the document content, and the system provides accurate answers along with source document references.

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import RetrievalQA
import torch
import os
import warnings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

warnings.filterwarnings("ignore")

**PDF Document Loading and Text Extraction:** Loaded PDF documents and split them into chunks for further processing and embedding using HuggingFace's embeddings.

In [ ]:
pdf_loader = PyPDFLoader('/content/1706.03762.pdf')
pages = pdf_loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 1024,
chunk_overlap  = 128,
length_function = len,
)
chunks = text_splitter.split_documents(pages)

In [ ]:
chunks[14].page_content

'extremely small gradients4. To counteract this effect, we scale the dot products by1√dk.\n3.2.2 Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values htimes with different, learned\nlinear projections to dk,dkanddvdimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products get large, assume that the components of qandkare independent random\nvariables with mean 0and variance 1. Then their dot product, q·k=Pdk\ni=1qiki, has mean 0and variance dk.\n4'

**Text Embedding and Vector Storage:**
Embedded text chunks using Sentence Transformers and stored them in a FAISS index for efficient similarity search and document retrieval.

In [ ]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                    model_kwargs={'device': 'cuda'})

store = FAISS.from_texts([str(chunk) for chunk in chunks], Embeddings)

**Question Answering (QA) System Setup:**
Configured a QA system using Mistral-7B-Instruct model for answering questions based on PDF document content, integrating FAISS for document retrieval.


In [ ]:
## Without LLM
question = """
what is the difference between supervised and unsupervised learning?
"""
docs = store.similarity_search(question, k = 2)

In [ ]:
prompt = """
    Using this information:
    \n
    Context: {context}
    \n
    Answer the following:
    \n
    Question: {question}
    \n
    Answer:\n
    """
prompt = ChatPromptTemplate.from_template(prompt)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config = bnb_config)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
hf = HuggingFacePipeline(pipeline=pipe)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=hf,
                                       retriever=store.as_retriever(search_kwargs={'k': 3}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

**Inference and Results Display:**
Conducted inference to answer user questions based on the stored PDF document content, displaying both the question and the model's generated answer along with source document references.

In [ ]:
question = "what is the transformer model architecture? "

result = qa_chain({"query": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result["query"]

'why attention is so important?'

In [ ]:
print(result["result"].split("Answer:")[1])



     Attention mechanisms are important in natural language processing models because they allow the model to focus on specific parts of the input sequence when generating an output. This is particularly useful in tasks such as machine translation, where the meaning of a word or phrase can depend on its context in the sentence. Attention mechanisms enable the model to dynamically weigh the importance of different parts of the input sequence when generating an output, which can lead to more accurate and interpretable models. In the given paper, the authors found that self-attention could yield more interpretable models and that individual attention heads learned to perform different tasks, many of which appeared to exhibit behavior related to the syntactic and semantic structure of the sentences. Additionally, separable convolutions, which are used in the model, decrease the complexity of the model to a level equal to the combination of a self-attention layer and a point-wise feed-for

In [ ]:
print(result["source_documents"][0].page_content)

page_content='between any two positions in the network. Convolutional layers are generally more expensive than\nrecurrent layers, by a factor of k. Separable convolutions [ 6], however, decrease the complexity\nconsiderably, to O(k·n·d+n·d2). Even with k=n, however, the complexity of a separable\nconvolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,\nthe approach we take in our model.\nAs side benefit, self-attention could yield more interpretable models. We inspect attention distributions\nfrom our models and present and discuss examples in the appendix. Not only do individual attention\nheads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\nand semantic structure of the sentences.\n5 Training\nThis section describes the training regime for our models.\n5.1 Training Data and Batching\nWe trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million' metadata={'

In [ ]:
print("Your Question: ", result["query"])
print()
print("Your answer: ", result["result"])
print()
print("Sources: ", result["source_documents"])

Your Question:  what is the transformer model architecture? 

Your answer:  Human: 
    Using this information:
    

    Context: page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected feed-forward network. We employ a residual connection [ 11] around each of\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\nLayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the mo

In [ ]:
pipe("what is the transformer model architecture? ")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'what is the transformer model architecture?  Question 2: What is the difference between a transformer model and a recurrent neural network (RNN)?\n\nAnswer:\n\nQuestion 1: A transformer model is a type of neural network architecture introduced by Vaswani et al. in the paper "Attention is All You Need" (2017). It is designed for handling long-range dependencies in sequences, which is a challenge for traditional recurrent neural networks (RNNs). The transformer model uses self-attention mechanisms to compute the relationships between different parts of a sequence, allowing the model to focus on relevant parts of the input when making predictions. The architecture consists of an encoder and decoder, each with multiple layers of self-attention and feed-forward neural networks.\n\nQuestion 2: The main difference between a transformer model and a recurrent neural network (RNN) lies in their underlying mechanisms for handling sequence data. RNNs process sequences by recur